## Poetry Classification

### Imports

In [1]:
import tensorflow as tf
import tokenizers as tk
import numpy as np
from tensorflow.keras.layers import GRU, Embedding, Dense, Input, Dropout, Bidirectional, BatchNormalization, Flatten, Reshape
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

### Tokenization

In [2]:
tokenizer = tk.CharacterTokenizer(clean = True)
tokenizer.process_data('meters/data.txt')
tokenizer.train()

Reading the data ...
Cleaning the data ...
Splitting the data ...
Training CharacterTokenizer ...


### Data

In [3]:
sentences = open('data/raw/train.txt', 'r').read().splitlines()
labels = open('meters/labels.txt', 'r').read().splitlines()

max_length = 100 

X = tokenizer.encode_sentences(sentences, max_length = max_length)
y = np.array([int(lbl) for lbl in labels])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [4]:
sentences[0]

'يا ضيف طيف ما هداه لمضجعي # إلا لهيب في الحشى يتوقد'

### Model

In [5]:
model = Sequential()
model.add(Input((100,)))
model.add(Embedding(tokenizer.vocab_size, 256))
model.add(Bidirectional(GRU(units = 256, return_sequences=True)))
model.add(Bidirectional(GRU(units = 256, return_sequences=True)))
model.add(Bidirectional(GRU(units = 256)))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(14, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [6]:
model.fit(X_train, y_train, validation_split = 0.1, epochs = 10, batch_size= 128, shuffle = True)

Epoch 1/10
222/222 [==============================] - 410s 2s/step - loss: 2.1945 - accuracy: 0.2110 - val_loss: 1.8659 - val_accuracy: 0.3056
Epoch 2/10
222/222 [==============================] - 397s 2s/step - loss: 1.6794 - accuracy: 0.3894 - val_loss: 1.4048 - val_accuracy: 0.4934
Epoch 3/10
222/222 [==============================] - 393s 2s/step - loss: 1.1040 - accuracy: 0.6277 - val_loss: 0.8356 - val_accuracy: 0.7255
Epoch 4/10
222/222 [==============================] - 422s 2s/step - loss: 0.6844 - accuracy: 0.7844 - val_loss: 0.5612 - val_accuracy: 0.8255
Epoch 5/10
222/222 [==============================] - 433s 2s/step - loss: 0.5033 - accuracy: 0.8510 - val_loss: 0.5096 - val_accuracy: 0.8521
Epoch 6/10
222/222 [==============================] - 408s 2s/step - loss: 0.3962 - accuracy: 0.8850 - val_loss: 0.4341 - val_accuracy: 0.8686
Epoch 7/10
222/222 [==============================] - 368s 2s/step - loss: 0.3194 - accuracy: 0.9081 - val_loss: 0.3918 - val_accuracy: 0.8908

### Test

In [7]:
label2name = ['saree', 'kamel', 'mutakareb', 'mutadarak', 'munsareh', 'madeed', 
              'mujtath', 'ramal', 'baseet', 'khafeef', 'taweel', 'wafer', 'hazaj', 'rajaz']

In [8]:
def classify(sentence):
    sequence = tokenizer.encode_sentences([sentence], max_length = max_length)
    pred = model.predict(sequence)[0]
    print(label2name[np.argmax(pred, 0).astype('int')], np.max(pred))

In [9]:
classify("ما تردون على هذا المحب # دائبا يشكو إليكم في الكتب")
classify("ولد الهدى فالكائنات ضياء # وفم الزمان تبسم وسناء")
classify(" لك يا منازل في القلوب منازل # أقفرت أنت وهن منك أواهل")

ramal 0.99864715
kamel 0.9625275
kamel 0.99522


In [10]:
classify("ما تردون على هذا المحب # دائبا يشكو إليكم في الكتب")
classify("ولد الهدى فالكائنات ضياء # وفم الزمان تبسم وسناء")
classify(" لك يا منازل في القلوب منازل # أقفرت أنت وهن منك أواهل")
classify("ومن لم يمت بالسيف مات بغيره # تعددت الأسباب والموت واحد")
classify("أنا النبي لا كذب # أنا ابن عبد المطلب")
classify("هذه دراهم اقفرت # أم ربور محتها الدهور")
classify("هزجنا في بواديكم # فأجزلتم عطايانا")
classify("بحر سريع ماله ساحل # مستفعلن مستفعلن فاعلن")
classify("مَا مَضَى فَاتَ وَالْمُؤَمَّلُ غَيْبٌ # وَلَكَ السَّاعَةُ الَّتِيْ أَنْتَ فِيْهَا")
classify("يا ليلُ الصبّ متى غدهُ # أقيامُ الساعة موعدهُ")

ramal 0.99864715
kamel 0.9625275
kamel 0.99522
taweel 0.99869907
munsareh 0.35370013
madeed 0.51401275
hazaj 0.98468786
saree 0.9669881
mutadarak 0.9865726
mutadarak 0.99643195
